# Evaluate NWM Short Range Forecasts Made for Current Time

**Author(s):** 

<ul style="line-height:1.5;">
<li>Nana Oye Djan <a href="mailto:ndjan@andrew.cmu.edu">(ndjan@andrew.cmu.edu)</a></li>
</ul>

**Last Updated:** 
17th July 2025

**Purpose:**

This notebook provides code to retrieve NOAA National Water Model Short Range data from Amazon Web Services (AWS) in netcdf format at the current time as well as retrieve instantaneous USGS gage data (discharge) for reaches in Travis County to evaluate the performance of the NWM forecasts made in previous initialization runs for the current time.

**Description:**

This notebook determines the reaches USGS gages are on in Travis County using OWP's <a href="https://github.com/NOAA-OWP/hydrotools">(hydrotools)</a></li>, uses the reach feature ids to download the short range forecast and uses the USGS site ids to get the instantaneous discharge readings for those sites. It then evaluates the performance of the NWM short range forecasts made in previous initialization runs for the current time using the Continuous Ranked Probability Score to evaluate the accuracy and precision of the short range forecasts and percentage bias to determine the degree of over or underestimation of the short range forecasts.

**Data Description:**

This notebook uses data developed and published by NOAA on Amazon Web Services (AWS) as described in detail in <a href="https://registry.opendata.aws/noaa-nwm-pds/">(this registry)</a></li> of open data entry. The National Water Model (NWM) is a water resources model that simulates and forecasts water budget variables, including snowpack, evapotranspiration, soil moisture and streamflow, over the entire continental United States (CONUS). It is operated by NOAA’s Office of Water Prediction. This bucket contains a four-week rollover of the Short Range Forecast model output and the corresponding forcing data for the model. The model is forced with meteorological data from the High Resolution Rapid Refresh (HRRR) and the Rapid Refresh (RAP) models. The Short Range Forecast configuration cycles hourly and produces hourly deterministic forecasts of streamflow and hydrologic states out to 18 hours. It also uses information on USGS reaches in Travis County provided <a href="http://www.hydroshare.org/resource/c95e654312204ce0b4d8e31e71cd4354">(here)</a></li>

**Software Requirements:**

This notebook uses Python v3.10.14 and requires the following specific Python libraries: 

> nest_asyncio: 1.6.0 \
   numpy: 1.26.4 \
   pandas: 2.3.0 \
   geopandas: 0.14.4 \
   dataretrieval: 1.0.12 \
   scores: 1.0.2 \
   xarray: 2025.4.0 \
   s3fs: 2025.5.1 \
   fsspec: 2025.5.1 \
   ipython: 8.37.0 \
   os: Python 3.10.14 (stdlib) \
   datetime / timedelta: Python 3.10.14 (stdlib) \
   re: Python 3.10.14 (stdlib) 
 

**Disclosure**
The code contained in this notebook was partially created and revised by ChatGPT, an AI language model developed by OpenAI

#### 1. Setup

In [71]:
#Import necessary packages
import nest_asyncio
nest_asyncio.apply()
import numpy as np
import pandas as pd
import geopandas as gpd
from hydrotools.nwm_client.NWMFileClient import NWMFileClient, QueryError, NWMClient
from dataretrieval import nwis
from scores.probability import crps_for_ensemble
from datetime import datetime, timedelta
import xarray as xr
import s3fs
from IPython.display import display
import fsspec
import os, re
from datetime import date

#### 2. Define Important Parameters and Functions

In [72]:
#S3 bucket info
s3_bucket = 'noaa-nwm-pds'
forecast_path = 'short_range'
variable = 'streamflow'
filename = 'channel_rt'
valid_time = datetime.utcnow()
max_lead_hours = 10
CFS_TO_M3S    = 0.028316846592  # ft³/s → m³/s

In [73]:
# URL builder function
def construct_s3_url(cycle_dt: datetime, lead_hr: int) -> str:
    """
    Build the S3 URL for:
      nwm.t{HH}z.short_range.{filename}.f{LLL}.conus.nc
    """
    date_str = cycle_dt.strftime("%Y%m%d")      # e.g. '20250708'
    cycle_hr = f"{cycle_dt.hour:02d}"           # e.g. '16'
    lead_str = f"{lead_hr:03d}"                 # e.g. '005'
    fname    = (
        f"nwm.t{cycle_hr}z.short_range."
        f"{filename}.f{lead_str}.conus.nc"
    )
    return f"s3://{s3_bucket}/nwm.{date_str}/{forecast_path}/{fname}"


#### 3. Crosswalk USGS -> NWM Feature IDS to subset gauges in Travis County

In [ ]:
#Load gauge shapefile and extract USGS IDs
stations = gpd.read_file(
    "/path/to/NWPS_Gages/NWPSGagesTravisCounty_All.shp"
)
stations["usgs_id"] = stations["usgs_id"].astype(str).str.zfill(8)
site_ids = stations["usgs_id"].tolist()

# Build the NWM ↔ USGS crosswalk
nwm_client = NWMFileClient()
raw_cw = (
    nwm_client.crosswalk
      .reset_index()
      .rename(columns={"index":"nwm_feature_id"})
      [["nwm_feature_id","usgs_site_code"]]
)
cw = raw_cw[raw_cw["usgs_site_code"].isin(site_ids)]
nwm_ids_int = cw["nwm_feature_id"].astype(int).tolist()

#### 4. Get SR forecasts made for current day and time

In [75]:
# initialize filesystem
fs = s3fs.S3FileSystem(anon=True, client_kwargs={"region_name":"us-east-1"})
today = datetime.utcnow().date()

In [76]:
#find most recent folder (up to 3 days back) with channel_rt files
valid_prefix = None
published_runs = set()
for delta in range(4):
    dt     = today - timedelta(days=delta)
    prefix = f"{s3_bucket}/nwm.{dt:%Y%m%d}/short_range/"
    try:
        all_files = fs.find(prefix)
    except Exception:
        continue

    runs = {
        int(m.group(1))
        for f in all_files
        if f.endswith(".conus.nc")
        and ".channel_rt." in f
        and (m := re.search(r"nwm\.t(\d{2})z", os.path.basename(f)))
    }
    if runs:
        valid_prefix   = prefix
        published_runs = runs
        break

if not valid_prefix:
    raise RuntimeError("No short_range/channel_rt folder in last 4 days")

# clear any cached listings so we see newly uploaded runs
fs.invalidate_cache(valid_prefix)

# rebuild channel_files with bare keys
channel_files = [
    f for f in fs.find(valid_prefix)
    if f.endswith(".conus.nc") and ".channel_rt." in f
]

print(f"Using folder {valid_prefix} with cycles: {sorted(published_runs)}Z")

# build a lookup of (run_hr, lead_hr) → bare S3 key
run_lead_map = {}
for path in channel_files:
    fn     = os.path.basename(path)
    m_run  = re.search(r"nwm\.t(\d{2})z", fn)
    m_lead = re.search(r"\.f(\d{3})\.", fn)
    if m_run and m_lead:
        run_hr, lead_hr = int(m_run.group(1)), int(m_lead.group(1))
        if 1 <= lead_hr <= max_lead_hours:
            run_lead_map[(run_hr, lead_hr)] = path  # <— no "s3://"

# parse the prefix date for full datetimes
date_str  = valid_prefix.split("/")[1].split(".")[1]  # e.g. "20250708"
prefix_dt = datetime.strptime(date_str, "%Y%m%d")

# Latest‐cycle forecasts (tXXZ → f001–f010)
latest_run_hr = max(run for run, _ in run_lead_map)
latest_run_dt = prefix_dt + timedelta(hours=latest_run_hr)

#comids_int       = [int(c) for c in comids]
records_current  = []
chosen_run_hr    = None

records_current = []
for run_hr in sorted(published_runs, reverse=True):
    run_dt = prefix_dt + timedelta(hours=run_hr)
    temp   = []
    for lead_hr in range(1, max_lead_hours+1):
        key = run_lead_map.get((run_hr, lead_hr))
        if not key:
            continue

        url = f's3://{key}'
        ds  = xr.open_dataset(
            url,
            engine='h5netcdf',
            storage_options={'anon':True}
        )

        # filter by your crosswalk feature_ids
        ds_ids      = ds['feature_id'].values.astype(int)
        present_ids = [fid for fid in nwm_ids_int if fid in ds_ids]
        if not present_ids:
            continue

        da = ds[variable].sel(feature_id=present_ids).load()
        df = da.to_dataframe().reset_index()
        df['feature_id'] = df['feature_id'].astype(int).astype(str)
        df = (
            df
            .set_index('feature_id')
            .reindex([str(fid) for fid in nwm_ids_int])
            .reset_index()
        )
        df['valid_time']   = run_dt + timedelta(hours=lead_hr)
        df['forecast_run'] = run_dt
        df['lead_hour']    = lead_hr
        temp.append(df)

    if temp:
        df_current = pd.concat(temp, ignore_index=True)
        print(f"✔ Using t{run_hr:02d}Z run for current forecasts")
        display(df_current[['feature_id','valid_time','lead_hour',variable]].head(5))
        break

# pick the valid_time closest to “now”
now = datetime.utcnow()
valid_time_current = min(
    df_current['valid_time'].unique(),
    key=lambda vt: abs(vt - now)
)
print(f"Ensemble valid_time chosen: {valid_time_current}")

#Pull the 10‐member ensemble for that single valid_time → df_ensemble_current

records_ensemble = []
for lead_hr in range(1, max_lead_hours+1):
    run_dt = valid_time_current - timedelta(hours=lead_hr)
    key    = run_lead_map.get((run_dt.hour, lead_hr))
    if not key:
        continue

    url = f's3://{key}'
    ds  = xr.open_dataset(
        url,
        engine='h5netcdf',
        storage_options={'anon':True}
    )

    ds_ids      = ds['feature_id'].values.astype(int)
    present_ids = [fid for fid in nwm_ids_int if fid in ds_ids]
    if not present_ids:
        continue

    da = ds[variable].sel(feature_id=present_ids).load()
    df = da.to_dataframe().reset_index()
    df['feature_id'] = df['feature_id'].astype(int).astype(str)
    df = (
        df
        .set_index('feature_id')
        .reindex([str(fid) for fid in nwm_ids_int])
        .reset_index()
    )
    df['valid_time']   = valid_time_current
    df['forecast_run'] = run_dt
    df['lead_hour']    = lead_hr
    records_ensemble.append(df)

df_ensemble_current = pd.concat(records_ensemble, ignore_index=True)
print("✔ Got 10-member ensemble for current time")
display(df_ensemble_current[['feature_id','forecast_run','lead_hour',variable]].head(10))

Using folder noaa-nwm-pds/nwm.20250714/short_range/ with cycles: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]Z
✔ Using t14Z run for current forecasts


,feature_id,valid_time,lead_hour,streamflow
0,5673157,2025-07-14 15:00:00,1,0.08
1,5781203,2025-07-14 15:00:00,1,0.08
2,5781223,2025-07-14 15:00:00,1,0.03
3,5781313,2025-07-14 15:00:00,1,0.32
4,5781319,2025-07-14 15:00:00,1,0.02


Ensemble valid_time chosen: 2025-07-14 16:00:00
✔ Got 10-member ensemble for current time


,feature_id,forecast_run,lead_hour,streamflow
0,5673157,2025-07-14 14:00:00,2,0.09
1,5781203,2025-07-14 14:00:00,2,0.09
2,5781223,2025-07-14 14:00:00,2,0.03
3,5781313,2025-07-14 14:00:00,2,0.32
4,5781319,2025-07-14 14:00:00,2,0.04
5,5781345,2025-07-14 14:00:00,2,0.25
6,5781709,2025-07-14 14:00:00,2,0.02
7,5781703,2025-07-14 14:00:00,2,0.03
8,5781731,2025-07-14 14:00:00,2,0.07
9,5781373,2025-07-14 14:00:00,2,0.07


#### 5. Get instantaneous gage data (discharge) for current time from USGS gages on NWM reaches

In [78]:
#Pull in USGS Discharge Data for today
parameterCode = "00060"   # discharge
today         = date.today().isoformat()   # e.g. "2025-07-09"

discharge_df, meta = nwis.get_iv(
    sites=site_ids,
    parameterCd=parameterCode,
    start=today,
    end=today
)

#keep only the first column (discharge column)
df_iv = discharge_df.iloc[:, :1]

# View
display(df_iv.head())

# keep only the discharge column and reset index (site_id and datetime columns were in a multi-index)
df_iv = discharge_df.iloc[:, :1].reset_index()
df_iv = df_iv.rename(columns={df_iv.columns[-1]: "discharge_obs_cfs"})

obs_current = (
    df_iv[["site_no", "discharge_obs_cfs"]]
      .drop_duplicates(subset="site_no")
      .rename(columns={"site_no": "usgs_site_code"})
)
#Convert from cfs to m3/s
obs_current["discharge_obs_m3s"] = obs_current["discharge_obs_cfs"] * CFS_TO_M3S

00060_15-minute updates
site_no  datetime                                          
08105872 2025-07-14 05:00:00+00:00                    213.0
         2025-07-14 05:05:00+00:00                    213.0
         2025-07-14 05:10:00+00:00                    213.0
         2025-07-14 05:15:00+00:00                    213.0
         2025-07-14 05:20:00+00:00                    213.0

#### 6. Determine CRPS and Percentage Bias for Reaches with Gage Data

In [ ]:
# Calculate short range ensemble mean
en_wide = (
    df_ensemble_current
      .pivot(index="feature_id", columns="lead_hour", values=variable)
      .rename_axis(columns=None)
      .reset_index()
)
member_cols = sorted(c for c in en_wide.columns if isinstance(c, int))
member_names = [f"m{c}" for c in member_cols]
en_wide = en_wide.rename(columns={c: f"m{c}" for c in member_cols})
en_wide["ensemble_mean_m3s"] = en_wide[member_names].mean(axis=1)

# unify crosswalk IDs as strings
cw_small = (
    cw[["nwm_feature_id", "usgs_site_code"]]
      .assign(nwm_feature_id=lambda df: df["nwm_feature_id"].astype(str))
      .rename(columns={"nwm_feature_id": "feature_id"})
)

# Ensure en_wide.feature_id is a string
en_wide['feature_id'] = en_wide['feature_id'].astype(str)

# Build cw_small with string IDs (as before)
cw_small = (
    cw[['nwm_feature_id','usgs_site_code']]
      .assign(nwm_feature_id=lambda df: df['nwm_feature_id'].astype(str))
      .rename(columns={'nwm_feature_id':'feature_id'})
)

# Merge
df = (
    en_wide
      .merge(cw_small, on='feature_id')
      .merge(obs_current, on='usgs_site_code')
)
df["bias_m3s"] = df["ensemble_mean_m3s"] - df["discharge_obs_m3s"]
df["pct_bias"] = df["bias_m3s"] / df["discharge_obs_m3s"] * 100

def compute_crps(row):
    fc = np.array([row[m] for m in member_names], dtype=float)
    da_fc = xr.DataArray(
        data=fc[np.newaxis, :],
        dims=["time", "ensemble_member"],
        coords={"time": [0], "ensemble_member": np.arange(fc.size)},
    )
    da_obs = xr.DataArray(
        data=[float(row["discharge_obs_m3s"])],
        dims=["time"],
        coords={"time": [0]},
    )
    return float(
        crps_for_ensemble(
            da_fc,
            da_obs,
            ensemble_member_dim="ensemble_member",
            method="ecdf",
        ).values
    )

df["crps_m3s"] = df.apply(compute_crps, axis=1)

# Results
result = df[
    [
        "usgs_site_code",
        "feature_id",
        "discharge_obs_m3s",
        "ensemble_mean_m3s",
        "bias_m3s",
        "pct_bias",
        "crps_m3s",
    ]
]
print(result.to_string(index=False))

usgs_site_code feature_id  discharge_obs_m3s  ensemble_mean_m3s  bias_m3s    pct_bias  crps_m3s
      08172400    1631087                NaN           0.526667       NaN         NaN       NaN
      08105888    5671187                NaN           9.906666       NaN         NaN       NaN
      08105883    5671189                NaN           7.287778       NaN         NaN       NaN
      08105872    5671619           6.031488           3.870000 -2.161488  -35.836734  1.723464
      08105886    5673157           0.219172           0.156667 -0.062506  -28.518979  0.039233
      08158700    5780099           0.175281           0.214444  0.039163   22.343036  0.026817
      08154700    5781189                NaN           1.627778       NaN         NaN       NaN
      08158380    5781203           0.043042           3.130000  3.086958 7172.033182  0.616094
      08158200    5781217                NaN           4.603333       NaN         NaN       NaN
      08156675    5781223               